**Optimizing HR by tuning the SF**

In order to get the optimal HR that we want to achive during a run for a specimen we need to first make him reach his maximum HR and than calcukating about 50-70% of it. Than when conducting the optimization experiment we can use the following "Optimal HR" to have a goal/wanted HR to achive during the run, and tuning using the SF.



**Module 1: Finding maximum HR **
Using a sport activity that will make the speciman reach very high HR and hopefully the maximum HR


**Module 2: Finding optimal HR **
need to complete


In [ ]:

from multiprocessing import Process,Value,Queue
from pydub import AudioSegment
import logging

"""
A scrit that allow real time musique speed modification with calcul on another Proccess to avoid overloading the main boucle
"""

# method from https://stackoverflow.com/questions/51434897/how-to-change-audio-playback-speed-using-pydub
def speed_change(sound:AudioSegment, speed=1.0):
    # Manually override the frame_rate. This tells the computer how many
    # samples to play per second
    sound_with_altered_frame_rate = sound._spawn(sound.raw_data, overrides={
        "frame_rate": int(sound.frame_rate * speed)
    })

    # convert the sound with altered frame rate to a standard frame rate
    # so that regular playback programs will work right. They often only
    # know how to play audio at standard frame rate (like 44.1k)
    return sound_with_altered_frame_rate.set_frame_rate(sound.frame_rate)

musique_original = AudioSegment.from_file("shit.wav")

sound_factor = Value('d',1)
sound_offset = 500

Sounds_buffer = Queue(maxsize=2) # maxsize will be interfering with the update (less = more reactive but less resilient to lag)

def generator(sound_buffer:Queue,factor:Value,offset:int=500):
    # var declaration
    musique_alternate = musique_original # current study of the musique
    player_offset = offset # offset of each sample
    max_occilation = 0.01 # max modification of speed between two sample (lesser => more faded)
    local_factor = factor.value # local factor of speed

    while True:
        if not Sounds_buffer.full(): # if the Queue is not full
            # if local factor need update
            if local_factor<factor.value:
                local_factor += max_occilation
            elif local_factor>factor.value:
                local_factor -= max_occilation

            if len(musique_alternate) > player_offset*local_factor: # check if we can take a normal sample in the study
                sample, musique_alternate = musique_alternate[:player_offset*local_factor], musique_alternate[player_offset*local_factor:]
            else: # else we just play what rest and loop the study back
                sample,musique_alternate = musique_alternate,musique_original

            _sample = speed_change(sample,local_factor)
            buffer = bytearray(_sample.raw_data)
            sound_buffer.put(buffer) # buffer send to the partaged Queue

p = Process(target=generator,args=(Sounds_buffer,sound_factor,sound_offset),daemon=True)

In [10]:
from audiotsm import phasevocoder
from audiotsm.io.wav import WavReader, WavWriter

with WavReader("shit.wav") as reader:
    with WavWriter("shit_out.wav", reader.channels, reader.samplerate) as writer:
        tsm = phasevocoder(reader.channels, speed=1.2)
        tsm.run(reader, writer)
        
        

In [13]:
import sounddevice as sd

# List available audio devices and their indices
print("hello")
print(sd.query_devices())

# Specify a valid device index explicitly
device_idx = 0  # Replace with the index of your desired output device
# sd.play(np.ascontiguousarray(writer.data.T), samplerate, device=device_idx, blocking=True)


hello



In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
sine
~~~~

Run a TSM procedure on a signal generated with numpy.
"""
# pylint: disable=invalid-name

import numpy as np
import sounddevice as sd
from audiotsm import wsola
from audiotsm.io.array import ArrayReader, ArrayWriter


# The parameters of the input signal
length = 1  # in seconds
samplerate = 44100  # in Hz
frequency = 440  # an A4

# Generate the input signal
time = np.linspace(0, length, int(length * samplerate))
input_signal = np.sin(np.pi * frequency * time).reshape((1, -1))

# Run the TSM procedure
reader = ArrayReader(input_signal)
writer = ArrayWriter(channels=1)

tsm = wsola(channels=1, speed=0.5)
tsm.run(reader, writer)

# Play the output
# This example was written to show how to use an ArrayWriter. If you want to
# play the output of a TSM procedure you should use an
# audiotsm.io.stream.StreamWriter.
sd.play(np.ascontiguousarray(writer.data.T), samplerate, blocking=True)